<a href="https://colab.research.google.com/github/123RohitVarshit/123RohitVarshit/blob/main/ETH_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/eth-lre/pedagogicalrl.git

Cloning into 'pedagogicalrl'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 114 (delta 41), reused 69 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (114/114), 672.30 KiB | 4.20 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [ ]:
%cd pedagogicalrl


/content/pedagogicalrl


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!git clone https://github.com/eth-lre/pedagogicalrl.git

Cloning into 'pedagogicalrl'...
remote: Enumerating objects: 114, done.
remote: Counting objects: 100% (114/114), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 114 (delta 41), reused 69 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (114/114), 672.30 KiB | 3.46 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [2]:
%cd /content/drive/MyDrive/pedagogicalrl

/content/drive/MyDrive/pedagogicalrl


In [3]:
# 1. Install ONLY lightweight tools (No heavy torch/cuda downloads)
!pip install hydra-core omegaconf python-dotenv openai google-generativeai colorama --quiet

# This code creates a fake vLLM library in memory that matches the repo's imports exactly.
import sys
import types
from dataclasses import dataclass
from typing import List, Any, Optional
from importlib.machinery import ModuleSpec

print("🛡️ Initializing Robust vLLM Mocking...")

# Helper to create valid modules with specs (Satisfies 'transformers' checks)
def mock_module(name):
    m = types.ModuleType(name)
    m.__spec__ = ModuleSpec(name=name, loader=None)
    sys.modules[name] = m
    return m

# --- A. Mock 'vllm' Top-Level ---
vllm = mock_module("vllm")

@dataclass
class SamplingParams:
    temperature: float = 0.7
    top_p: float = 1.0
    top_k: int = -1
    max_tokens: int = 100
    n: int = 1
    logits_processors: Any = None
    stop: Optional[List[str]] = None

@dataclass
class CompletionOutput:
    index: int
    text: str
    token_ids: List[int]
    cumulative_logprob: float
    logprobs: List[Any]

@dataclass
class RequestOutput:
    request_id: str
    prompt: str
    outputs: List[CompletionOutput]
    prompt_token_ids: List[int]
    prompt_logprobs: List[Any]
    finished: bool

class PoolingOutput:
    pass

class LLM:
    def __init__(self, *args, **kwargs): pass
    def encode(self, *args, **kwargs): return []
    def chat(self, *args, **kwargs): return []

vllm.SamplingParams = SamplingParams
vllm.CompletionOutput = CompletionOutput
vllm.RequestOutput = RequestOutput
vllm.PoolingOutput = PoolingOutput
vllm.LLM = LLM

# --- B. Mock 'vllm.config' (for pedagogical_reward.py) ---
vllm_config = mock_module("vllm.config")

class PoolerConfig:
    def __init__(self, pooling_type, **kwargs): pass

vllm_config.PoolerConfig = PoolerConfig
vllm.config = vllm_config

# --- C. Mock 'vllm.distributed.parallel_state' (for data_parallel_vllm.py) ---
vllm_dist = mock_module("vllm.distributed")
vllm_dist_ps = mock_module("vllm.distributed.parallel_state")

def dummy_destroy(): pass
vllm_dist_ps.destroy_model_parallel = dummy_destroy
vllm_dist_ps.destroy_distributed_environment = dummy_destroy

vllm_dist.parallel_state = vllm_dist_ps
vllm.distributed = vllm_dist

# --- D. Mock 'liger_kernel' & 'deepspeed' (for trainer imports) ---
mock_module("liger_kernel")
mock_module("liger_kernel.chunked_loss")
mock_module("deepspeed")

print("✅ vLLM, DeepSpeed, and Liger Kernel successfully mocked.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 6.6 MB/s eta 0:00:00
🛡️ Initializing Robust vLLM Mocking...
✅ vLLM, DeepSpeed, and Liger Kernel successfully mocked.


In [4]:
import os

# 1. TOPOLOGY LOGIC (Genotype)
code_topology = """
import random
from dataclasses import dataclass
from typing import List

class Action:
    DIAGNOSE = "diagnose"
    SCAFFOLD = "scaffold"
    HINT = "hint"
    VERIFY = "verify"
    ENCOURAGE = "encourage"

    @staticmethod
    def all(): return [Action.DIAGNOSE, Action.SCAFFOLD, Action.HINT, Action.VERIFY, Action.ENCOURAGE]

@dataclass
class Topology:
    genes: List[str]
    fitness: float = -999.0

    def get_instruction(self, turn_idx: int) -> str:
        if turn_idx >= len(self.genes): return "Guide the student gently."
        step = self.genes[turn_idx]
        prompts = {
            Action.DIAGNOSE: "Do NOT solve. Ask the student what they think the first step is.",
            Action.SCAFFOLD: "Break the problem down. Create a simpler example with different numbers.",
            Action.HINT: "Give a conceptual hint about the formula, but do NOT mention the numbers.",
            Action.VERIFY: "Ask the student to double-check their last calculation.",
            Action.ENCOURAGE: "Tell them they are making progress, but ask them to try the step again."
        }
        return prompts.get(step, "Guide the student.")

    def mutate(self):
        idx = random.randint(0, len(self.genes)-1)
        self.genes[idx] = random.choice(Action.all())

    @classmethod
    def crossover(cls, p1, p2):
        if len(p1.genes) < 2: return cls(genes=p1.genes)
        split = random.randint(1, len(p1.genes)-1)
        return cls(genes=p1.genes[:split] + p2.genes[split:])

    @classmethod
    def random_init(cls, length=4):
        return cls(genes=random.choices(Action.all(), k=length))
"""
with open("src/topology.py", "w") as f: f.write(code_topology)

# 2. CLASSROOM WRAPPER (Phenotype)
code_top_class = """
from src.classroom import Conversation, ConversationState
from src.topology import Topology
from jinja2 import Template

class TopologyConversation(Conversation):
    def __init__(self, problem, answer, generation_cfg, topology: Topology):
        super().__init__(problem, answer, generation_cfg)
        self.topology = topology
        self.turn_count = 0
        self.template = Template(
            "TASK: Math Tutor.\\n"
            "STRATEGY: {{instruction}}\\n"
            "PROBLEM: {{problem}}\\n"
            "HISTORY: {{history}}\\n"
            "RESPONSE:"
        )

    def get_conversation(self):
        if self.state == ConversationState.TEACHER_TURN:
            instruction = self.topology.get_instruction(self.turn_count)
            hist = "\\n".join([f"{m['role'].upper()}: {m['content']}" for m in self.conversation[-4:]])
            prompt = self.template.render(problem=self.problem, instruction=instruction, history=hist)
            self.turn_count += 1
            return [{"role": "user", "content": prompt}]
        return super().get_conversation()
"""
with open("src/topology_classroom.py", "w") as f: f.write(code_top_class)

# 3. EVOLUTION RUNNER (Main Script)
code_runner = """
# RE-INJECT MOCKS FOR THE SCRIPT PROCESS
import sys, types
from dataclasses import dataclass
from typing import List, Any
from importlib.machinery import ModuleSpec

def mock_module(name):
    m = types.ModuleType(name)
    m.__spec__ = ModuleSpec(name=name, loader=None)
    sys.modules[name] = m
    return m

if "vllm" not in sys.modules:
    vllm = mock_module("vllm")
    @dataclass
    class SamplingParams:
        temperature: float = 0.7; top_p: float = 1.0; top_k: int = -1; max_tokens: int = 100; n: int = 1; logits_processors: Any = None; stop: Any = None
    @dataclass
    class CompletionOutput:
        index: int; text: str; token_ids: List[int]; cumulative_logprob: float; logprobs: List[Any]
    @dataclass
    class RequestOutput:
        request_id: str; prompt: str; outputs: List[CompletionOutput]; prompt_token_ids: List[int]; prompt_logprobs: List[Any]; finished: bool
    class PoolingOutput: pass
    class LLM:
        def __init__(self, *args, **kwargs): pass
        def encode(self, *args, **kwargs): return []
        def chat(self, *args, **kwargs): return []

    vllm.SamplingParams = SamplingParams; vllm.CompletionOutput = CompletionOutput
    vllm.RequestOutput = RequestOutput; vllm.PoolingOutput = PoolingOutput; vllm.LLM = LLM

    vllm_config = mock_module("vllm.config")
    class PoolerConfig:
        def __init__(self, pooling_type, **kwargs): pass
    vllm_config.PoolerConfig = PoolerConfig
    vllm.config = vllm_config

    vllm_dist = mock_module("vllm.distributed")
    vllm_dist_ps = mock_module("vllm.distributed.parallel_state")
    vllm_dist_ps.destroy_model_parallel = lambda: None
    vllm_dist_ps.destroy_distributed_environment = lambda: None
    vllm_dist.parallel_state = vllm_dist_ps
    vllm.distributed = vllm_dist

    mock_module("liger_kernel")
    mock_module("liger_kernel.chunked_loss")
    mock_module("deepspeed")

# --- ACTUAL SCRIPT STARTS HERE ---
import hydra
import random
import copy
import logging
import os
from dotenv import load_dotenv

from src.classroom import Classroom, ConversationState
from src.topology import Topology, Action
from src.topology_classroom import TopologyConversation
from config.eval import EvalConfig
from hydra.core.config_store import ConfigStore

logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger()

cs = ConfigStore.instance()
cs.store(name="config", node=EvalConfig)

def calculate_fitness(conv, ans, length):
    hist = conv.conversation
    st_msgs = [m['content'] for m in hist if m['role'] == 'student']
    te_msgs = [m['content'] for m in hist if m['role'] == 'teacher']

    if not st_msgs: return 0.0

    success = str(ans) in st_msgs[-1]
    score = 100.0 if success else 10.0
    if success: score += (length - len(te_msgs)) * 15
    for m in te_msgs:
        if str(ans) in m: score -= 50.0
    return score

@hydra.main(config_path="config/eval", version_base=None)
def main(cfg: EvalConfig):
    load_dotenv()
    print("\\n🧬 INITIALIZING EVOLUTIONARY TOPOLOGIES (SECURE MODE)...")
    print("="*60)

    POP_SIZE = 4
    GENERATIONS = 3
    PROBLEM = "Solve for x: 3x + 12 = 27"
    ANSWER = "5"

    classroom = Classroom(cfg.student_model, cfg.teacher_model, cfg.judge_model, cfg.reward_model, cfg.generation, None)

    pop = [Topology.random_init() for _ in range(POP_SIZE)]
    best_score = -999

    for gen in range(GENERATIONS):
        print(f"\\n⚡ GENERATION {gen+1}/{GENERATIONS}")
        conversations = []
        for dna in pop:
            c = TopologyConversation(PROBLEM, ANSWER, cfg.generation, topology=dna)
            c.start_conversation()
            conversations.append(c)

        for _ in range(4):
            act_t = [c for c in conversations if c.state == ConversationState.TEACHER_TURN]
            if act_t: classroom.generate_next_teacher_utterances(act_t)
            act_s = [c for c in conversations if c.state == ConversationState.STUDENT_TURN]
            if act_s: classroom.generate_next_student_utterances(act_s)

        scores = []
        for i, c in enumerate(conversations):
            f = calculate_fitness(c, ANSWER, 4)
            pop[i].fitness = f
            scores.append(f)
            status = "✅ Solved" if f > 50 else "❌ Failed"
            print(f"   [Org {i}] {pop[i].genes} | Score: {f:.1f}")

        if max(scores) > best_score:
            best_score = max(scores)

        new_pop = []
        while len(new_pop) < POP_SIZE:
            p = random.choice(pop)
            child = copy.deepcopy(p)
            if random.random() < 0.6: child.mutate()
            new_pop.append(child)
        pop = new_pop

    print("\\n" + "="*60)
    print("🏆 EVOLUTION COMPLETE")
    print(f"Final Best Score: {best_score}")
    print("="*60)

if __name__ == "__main__":
    main()
"""
with open("run_evolution.py", "w") as f: f.write(code_runner)

print("✅ Research files created successfully.")

✅ Research files created successfully.


In [46]:
import os

classroom_path = "src/classroom.py"

print(f"🔧 Repairing {classroom_path}...")

with open(classroom_path, "r") as f:
    content = f.read()

# FIX 1: Replace the broken "self.getattr(generation_cfg" with "getattr(self.generation_cfg"
# This handles the specific error you saw in the traceback.
if "self.getattr(generation_cfg" in content:
    content = content.replace("self.getattr(generation_cfg", "getattr(self.generation_cfg")
    print("   -> Fixed 'self.getattr(generation_cfg' instances.")

# FIX 2: Catch cases where it might be "self.getattr(self.generation_cfg" (rare but possible artifact)
if "self.getattr(self.generation_cfg" in content:
    content = content.replace("self.getattr(self.generation_cfg", "getattr(self.generation_cfg")
    print("   -> Fixed 'self.getattr(self.generation_cfg' instances.")

# FIX 3: Ensure getattr is used correctly for 'max_turns' which might appear as self.getattr(generation_cfg...
# We simply do a global replace for the pattern caused by the previous regex script.
content = content.replace("self.getattr(", "getattr(self.")

# FIX 4: Re-verify local variables vs self attributes
# If the previous script replaced 'generation_cfg.attr' (local var) with 'getattr(generation_cfg, ...)'
# that is valid. We only want to remove 'self.' from BEFORE getattr.
# The replace above (FIX 3) might have been too aggressive if 'self.getattr' wasn't followed by 'generation_cfg'.
# Let's check specifically for the pattern in the traceback.

# Reload content to be clean and do precise replacement
with open(classroom_path, "r") as f:
    content = f.read()

# Precise Fixes based on your traceback
# The bad pattern: self.getattr(generation_cfg, 'max_tokens_in_conversation', 8192)
# The wanted pattern: getattr(self.generation_cfg, 'max_tokens_in_conversation', 8192)

patched_content = content.replace("self.getattr(generation_cfg", "getattr(self.generation_cfg")

# Write back
with open(classroom_path, "w") as f:
    f.write(patched_content)
    f.flush()
    os.fsync(f.fileno())

print("✅ File repaired. The AttributeError should be resolved.")

🔧 Repairing src/classroom.py...
   -> Fixed 'self.getattr(generation_cfg' instances.
✅ File repaired. The AttributeError should be resolved.


In [47]:
import os
from google.colab import userdata

# 1. Load Secrets Securely
try:
    # Try OpenRouter first
    os.environ["OPENROUTER_API_KEY"] = userdata.get('OPENROUTER_API_KEY')
    print("✅ Loaded OPENROUTER_API_KEY")
except:
    try:
        # Try Gemini
        os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')
        print("✅ Loaded GEMINI_API_KEY")
    except:
        print("❌ ERROR: Keys not found. Please set secrets in Colab sidebar.")

# 2. Run
!python run_evolution.py \
  --config-name Qwen2.5-7B-Instruct.yaml \
  teacher_model.use_openrouter=True \
  teacher_model.model_name_or_path="meta-llama/llama-3.1-8b-instruct" \
  +student_model.use_openrouter=True \
  student_model.model_name_or_path="meta-llama/llama-3.1-8b-instruct" \
  +judge_model.use_openrouter=True \
  judge_model.model_name_or_path="meta-llama/llama-3.1-8b-instruct" \
  +generation.number_judge_attempts=0

✅ Loaded OPENROUTER_API_KEY

🧬 INITIALIZING EVOLUTIONARY TOPOLOGIES (SECURE MODE)...

⚡ GENERATION 1/3
[2025-12-14 18:25:36,063][httpx][INFO] - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 18:25:36,107][httpx][INFO] - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 18:25:36,251][httpx][INFO] - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 18:25:36,383][httpx][INFO] - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
Attempt 1 succeeded
Attempt 1 succeeded
Attempt 1 succeeded
Attempt 1 succeeded
[2025-12-14 18:25:40,925][httpx][INFO] - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 18:25:41,269][httpx][INFO] - HTTP Request: POST https://openrouter.ai/api/v1/chat/completions "HTTP/1.1 200 OK"
[2025-12-14 18:25:41,329][httpx][INFO] - HTTP Request: POST https://openr